In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from termcolor import colored
import urllib.request
import os
import time
import uuid

In [2]:
service = Service("chromedriver_win64\chromedriver.exe")
options = Options()
# options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
driver = webdriver.Chrome(service=service, options=options)
    

In [3]:
from dotenv import load_dotenv
load_dotenv()
usr = os.getenv("USERNAME")
pswd = os.getenv("PASSWORD")

In [4]:
login_page = "https://www.instagram.com/accounts/login/"
driver.get(login_page)
time.sleep(4)

# Accept the website cookies
# driver.find_element(By.XPATH, "/html/body/div[4]/div/div/button[1]").click()
# time.sleep(3)

# Login with a random account since we can't scrap stories without being logged
driver.find_element(By.NAME, "username").send_keys(usr)
driver.find_element(By.NAME, "password").send_keys(pswd)

# Click the login button
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div').click()
print(colored("\n[SUCCESS]: Logged into the website. \n", "green"))
time.sleep(5)


[SUCCESS]: Logged into the website. 



In [11]:
# username = 'movieseriesquotess'
username = 'fcbarcelona'
story_link = "https://www.instagram.com/stories/{}/".format(username)

# Get access to the story link
driver.get(story_link)
time.sleep(1)

rows = []
print(colored("\n[SUCCESS]: Got into the story link. \n", "green"))
buttons = driver.find_elements(By.CSS_SELECTOR, 'div[role="button"]')
for button in buttons:
    if button.get_attribute('innerHTML') == 'View story':
        button.click()
        break  


[SUCCESS]: Got into the story link. 



In [12]:
while driver.current_url != "https://www.instagram.com/":
    # Collect the link to the video content of the story if it exists, otherwise take the image link
    
    is_video = False
    try:
        content_element = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/section/div[1]/div/div/div[1]/div[2]/div[1]/div/img')
        content_link = content_element.get_attribute('src')
        
    except:
        is_video = True
        probable_elements = driver.find_elements(By.CSS_SELECTOR, 'video')
        for element in probable_elements:
            print(element.get_attribute('innerHTML'))
        content_link = probable_elements[0].get_attribute('src')
        
    print(colored(f"\n[SUCCESS]: Got the content link:\n\t{content_link}. \n", "green"))

    # Get the link of the story
    insta_link = driver.current_url

    # Get the date of the story
    date = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/section/div[1]/div/div/div[1]/div[1]/div[2]/div[1]/div/div/div/div/span/time') \
                    .get_attribute("datetime")

    # Append all collected information into a row
    rows.append(
        {
            'Instagram URL': insta_link,
            'Content URL': content_link,
            'Date': date,
            'is_video': is_video
        }
    )

    # Click on the next button
    try:
        buttons = driver.find_elements(By.CSS_SELECTOR, 'div[role="button"]')
        for button in buttons:
            if button.get_attribute('innerHTML').find('Next') != -1:
                button.click()
                break
    except:
        break

    time.sleep(1)
    
# Save scrapped data into a dataframe
df = pd.DataFrame(rows)
tmp = os.path.join("collected_data", "pickles")
if not os.path.exists(tmp): os.makedirs(tmp)
df.to_pickle(os.path.join(tmp, "stories_{}.pkl".format(username)))
# driver.quit()
print(colored("\n[SUCCESS]: Scrapped all stories for the last 24h. \n", "green"))



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/10fec3da-1f02-41da-a071-3d66f7c59c91. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/61f907a3-8823-4b5e-82da-e2c3458cc5c3. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/0e73b03d-0509-4c6b-932c-9f7926bdd6c7. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/fb817f45-9da9-447b-9e22-ef6cf52db2be. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/16a9f8b8-8a33-49e8-a5b6-05632095d423. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/f9168b8e-0e58-4b9a-90ba-48a2505d98e5. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/b08484f0-f070-4d79-b0d3-b68999ce4628. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/b79c7e15-2800-4e51-83a8-14f6c1566a49. 



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/c302f245-e55c-448f-b6ef-7d5388ef9fda. 



[SUCCESS]: Got the content link:
	b

In [15]:
rows

[{'Instagram URL': 'https://www.instagram.com/stories/fcbarcelona/',
  'Content URL': 'blob:https://www.instagram.com/10fec3da-1f02-41da-a071-3d66f7c59c91',
  'Date': '2024-02-02T11:28:04.000Z',
  'is_video': True},
 {'Instagram URL': 'https://www.instagram.com/stories/fcbarcelona/3293815461102389544/',
  'Content URL': 'blob:https://www.instagram.com/61f907a3-8823-4b5e-82da-e2c3458cc5c3',
  'Date': '2024-02-02T11:30:17.000Z',
  'is_video': True},
 {'Instagram URL': 'https://www.instagram.com/stories/fcbarcelona/3293959963245949480/',
  'Content URL': 'blob:https://www.instagram.com/0e73b03d-0509-4c6b-932c-9f7926bdd6c7',
  'Date': '2024-02-02T16:17:25.000Z',
  'is_video': True},
 {'Instagram URL': 'https://www.instagram.com/stories/fcbarcelona/3293961048983534432/',
  'Content URL': 'blob:https://www.instagram.com/fb817f45-9da9-447b-9e22-ef6cf52db2be',
  'Date': '2024-02-02T16:19:34.000Z',
  'is_video': True},
 {'Instagram URL': 'https://www.instagram.com/stories/fcbarcelona/3293962819

In [16]:
for row in rows:
    content_link = row['Content URL']
    filename = f"{uuid.uuid4()}.jpg"
    dir = os.path.join("collected_data", username)
    if not os.path.exists(dir): os.makedirs(dir)
    try:
        urllib.request.urlretrieve(content_link, os.path.join(dir, filename))
        print(colored(f"\n[SUCCESS]: Downloaded the content link:\n\t{content_link}. \n", "green"))
        print(colored(f"\n[SUCCESS]: Saved the content to the file:\n\t{filename}. \n", "green"))
    except Exception as e:
        print(colored(f"\n[ERROR]: Couldn't download the content link:\n\t{content_link}. \n", "red"))
        print(e)


[ERROR]: Couldn't download the content link:
	blob:https://www.instagram.com/10fec3da-1f02-41da-a071-3d66f7c59c91. 

<urlopen error unknown url type: blob>

[ERROR]: Couldn't download the content link:
	blob:https://www.instagram.com/61f907a3-8823-4b5e-82da-e2c3458cc5c3. 

<urlopen error unknown url type: blob>

[ERROR]: Couldn't download the content link:
	blob:https://www.instagram.com/0e73b03d-0509-4c6b-932c-9f7926bdd6c7. 

<urlopen error unknown url type: blob>

[ERROR]: Couldn't download the content link:
	blob:https://www.instagram.com/fb817f45-9da9-447b-9e22-ef6cf52db2be. 

<urlopen error unknown url type: blob>

[ERROR]: Couldn't download the content link:
	blob:https://www.instagram.com/16a9f8b8-8a33-49e8-a5b6-05632095d423. 

<urlopen error unknown url type: blob>

[ERROR]: Couldn't download the content link:
	blob:https://www.instagram.com/f9168b8e-0e58-4b9a-90ba-48a2505d98e5. 

<urlopen error unknown url type: blob>

[ERROR]: Couldn't download the content link:
	blob:https: